# Building Q-Table for Choosing Cards

In [19]:
from card_choice_gym import CardChoiceEnv
from utils import save_q_table, write_calls
import numpy as np
from keras.models import load_model
import time
import os
from collections import Counter
from tqdm import tqdm

In [2]:
q = np.load('../models/q-table.npy')

In [2]:
if os.path.exists('../models/calling_nn_1.pickle'):
    cm = load_model('../models/calling_nn_1.pickle')


2022-05-15 18:23:33.627917: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
def LearnJoker(q_in=np.zeros((4, 10, 3, 4, 19, 36)), alpha_in=0.01, epsilon_in=0.5, gamma_in=0.95, episodes_in=100):
  env = CardChoiceEnv()

  alpha, gamma, epsilon = alpha_in, gamma_in, epsilon_in
  q = q_in
  
  wins = []
  good_calls = np.zeros((7,))

  actions = []

  for i in tqdm(range(episodes_in)): 
    done = False
    s = env.reset()
    env.calling_model = cm
    action_value_list= q[s]

    while True: 
      if np.random.random() < epsilon:
        action = env.action_space.sample()
        if not env.in_playable(0, action):
          action = env.action_space.sample()
      else:
        for poss, _ in Counter({a : b for (a, b) in enumerate(action_value_list)}).most_common():
          if env.in_playable(0, poss):
            action = poss 
    
      s_, r, done, _ = env.step(action)
      
      td_target = r + gamma * np.argmax(q[s_])
      td_error = td_target - q[s][action]
      s = s_

      q[s][action] += alpha * td_error
      if done:
        if r > 0:
          wins.append(i)
          good_calls = np.vstack((good_calls, [env.call_state]))

        break
      actions.append(action)
  return wins, good_calls, q, actions

In [35]:
%reload_ext autoreload
%autoreload 2

In [37]:
eps = 2000
wins, calls, q, acts= LearnJoker(epsilon_in=.5, episodes_in=eps, alpha_in=0.001)
print(f"Win rate: {len(wins)/ eps}")


100%|██████████| 2000/2000 [00:05<00:00, 348.62it/s]

Win rate: 0.2535
Time: 5.739034016000005 seconds


In [38]:
start = time.perf_counter()
eps = 1000
wins, calls, q, actions = LearnJoker(epsilon_in=0, episodes_in=eps, q_in=q)
print(f"Win rate: {len(wins)/eps}")
end = time.perf_counter()
print(f"Time: {end - start} seconds")


100%|██████████| 1000/1000 [00:03<00:00, 323.49it/s]

Win rate: 0.18
Time: 3.0932483799999773 seconds


In [28]:
from more_itertools import quantify
for i in range(36):
    print(acts.count(i))

4162
3587
4085
3550
4092
4067
4165
4231
4281
4167
4208
4270
4238
4222
4169
4236
4444
4203
4260
4279
4524
4298
4421
4481
4590
4432
4588
4520
4725
4695
5024
5050
5270
5280
5479
5707


In [34]:
save_q_table(q)
write_calls(calls)

NameError: name 'save_q_table' is not defined

In [23]:
Counter({"Quail": 5, 
        "Sam": 2}).most_common()

[('Quail', 5), ('Sam', 2)]